# Exemplos de queries SQL integrados ao Python e Pandas
Usaremos dados de exemplo do site [MySQL Tutorial](http://www.mysqltutorial.org/mysql-sample-database.aspx).

Para recriar o banco de dados em SQLite, sem Python, basta executar o arquivo com os comandos SQL na linha de comando do sistema operacional:

```shell
sqlite3 database.db < sample-database-dump.sql
```

O arquivo `database.db` será criado e tabelas e dados conforme especificados em `sql-sample-database.sql` serão populados.

Se estivéssemos lidando com um SGBD (sistema gerenciador de banco de dados) mais robusto, como MariaDB/MySQL, Oracle ou DB2, o comando acima seria diferente e deverá conter:

* o hostname ou IP do servidor de banco de dados
* usuário e senha de acesso
* nome do banco de dados em que vamos operar

Mas SQLite é bem mais simples e didático e opera num arquivo local.

## Visualizando o banco de dados, suas tabelas e dados

![database example](http://www.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

Cada SGBD tem seu próprio ferramental para navegar nos dados. MariaDB/MySQL tem o popular [PHP MyAdmin](https://www.phpmyadmin.net), Oracle e DB2 tem suas próprias ferramentas proprietárias, SQLite tem [DB Browser for SQLite](https://sqlitebrowser.org) e uma ferramenta online chamada [SQLite Online](https://sqliteonline.com).

Para navegar nos dados do DB SQLite que acabamos de criar, use uma das 3 opções:

* Use o comando `sqlite3`.
* Instale o **DB Browser for SQLite** e abra o arquivo.
* Envie o arquivo para o site **SQLiteOnline.com**.

## Anatomia de um SELECT

![img/select-query-anatomy.svg](img/select-query-anatomy.svg)

## SQLite

Vamos primeiro usar a estrutura e dados do arquivo `sql-sample-database.sql` para criar um banco de dados no arquivo `database.db`.

In [ ]:
import pandas as pd
import sqlite3

db = sqlite3.connect('database.db')

In [ ]:
script = 'sample-database-dump.sql'

db.cursor().executescript(open(script).read())

## SQLite com Pandas
Agora começamos a fazer queries:

In [ ]:
query = "SELECT * FROM productlines;"

queryResult = pd.read_sql_query(query,db)

queryResult

Lista os objetos (tabelas, índices etc) do DB

In [ ]:
query = "SELECT * FROM sqlite_master;"

queryResult = pd.read_sql_query(query,db)

queryResult

## Lista funcionários por chefe

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   employees e1,
   employees e2

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   e1.reportsTo = e2.employeeNumber

MOSTRE AS COLUNAS
   e2.firstName||' '||e2.lastName AS boss,
   e2.jobTitle                    AS department,
   e1.employeeNumber              AS ID,
   e1.firstName||' '||e1.lastName AS employee

ORDENE POR
   boss
```

In [ ]:
query = """SELECT
    e2.firstName||' '||e2.lastName AS boss,
    e2.jobTitle                    AS department,
    e1.employeeNumber              AS ID,
    e1.firstName||' '||e1.lastName AS employee
    
    FROM employees e1, employees e2
    
    WHERE e1.reportsTo=e2.employeeNumber
    
    ORDER BY boss;"""

queryResult = pd.read_sql_query(query,db)

queryResult

## Inspeciona pedidos de compra

Dados sobre pedidos aparecem nas tabelas `orders` e `orderdetails` e o que relaciona elas é a coluna `orderNumber`. Vamos inspecionar um único pedido.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
   orderNumber=10103

MOSTRE AS COLUNAS
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal
    
ORDENE POR
   orderLineNumber
```

In [ ]:
query = """SELECT
    orderLineNumber,
    productCode,
    priceEach,
    quantityOrdered,
    priceEach*quantityOrdered as priceTotal
    
    FROM orderdetails
    
    WHERE orderNumber=10103
    
    ORDER BY orderLineNumber;"""

queryResult = pd.read_sql_query(query,db)

queryResult

Vamos melhorar a leitura dessa tabela adicionando a descrição do produto. Faremos um **JOIN** com a tabela `products`.

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
   orderdetails AS o,
   products AS p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode AND
    orderNumber=10103

MOSTRE AS COLUNAS
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    
ORDENE POR
   orderLineNumber
```

In [ ]:
query = """SELECT
    o.orderLineNumber,
    o.priceEach,
    o.quantityOrdered,
    o.priceEach*o.quantityOrdered as itemTotal,
    o.productCode,
    p.productName
    
    FROM orderdetails AS o, products AS p
    
    WHERE
    o.productCode=p.productCode AND
    orderNumber=10103
    
    ORDER BY orderLineNumber;"""

queryResult = pd.read_sql_query(query,db)

queryResult

## Cálculo de totais e subtotais

Qual é o valor total de um pedido? Usaremos funções de agregação para obter esse valor.

In [ ]:
query = """SELECT
    o.orderNumber,
    sum(o.priceEach*o.quantityOrdered) AS orderTotal
    
    FROM orderdetails AS o
    
    WHERE
    o.orderNumber=10103;"""

queryResult = pd.read_sql_query(query,db)

queryResult

Total de todos os pedidos

In [ ]:
query = """SELECT
    o.orderNumber,
    count(o.orderNumber) as numberOfItems,
    sum(o.priceEach*o.quantityOrdered) AS orderTotal
    
    FROM orderdetails AS o
    
    GROUP BY o.orderNumber"""

queryResult = pd.read_sql_query(query,db)

queryResult

Total de todos os pedidos cujo total é maior que um certo valor

In [ ]:
query = """SELECT
    o.orderNumber,
    count(o.orderNumber) as numberOfItems,
    sum(o.priceEach*o.quantityOrdered) AS orderTotal
    
    FROM orderdetails AS o
    
    GROUP BY o.orderNumber
    
    HAVING orderTotal > 57000
    
    ORDER BY orderTotal DESC
"""

queryResult = pd.read_sql_query(query,db)

queryResult

Performance de venda por categoria de produtos

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productLine

ORDENE INVERSAMENTE POR
   lineTotal
```

In [ ]:
query = """SELECT

    p.productLine,
    count(p.productLine) AS nItems,
    sum(o.priceEach*o.quantityOrdered)/count(p.productLine) AS averagePerItem,
    sum(o.priceEach*o.quantityOrdered) AS lineTotal

    FROM orderdetails o, products p ON o.productCode=p.productCode
    
    GROUP BY p.productLine
    
    ORDER BY lineTotal DESC"""

queryResult = pd.read_sql_query(query,db)

queryResult

Qual é o produto que mais vende ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails o,
    products p

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    o.productCode=p.productCode

MOSTRE AS COLUNAS
    p.productName,
    count(o.quantityOrdered) AS nItems,
#    sum(o.priceEach*o.quantityOrdered) AS lineTotal

AGRUPE POR
    p.productCode

ORDENE INVERSAMENTE POR
   lineTotal
```

In [ ]:
query = """
SELECT
    p.productName,
    count(o.quantityOrdered) AS nItems
FROM
    orderdetails o,
    products p
WHERE 
    o.productCode=p.productCode

GROUP BY
    p.productCode

ORDER BY nItems ASC

LIMIT 8
"""

queryResult = pd.read_sql_query(query,db)

queryResult

Qual é o cliente que mais compra ?

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,
    customers as c

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber AND
    c.customerNumber = o.customerNumber

MOSTRE AS COLUNAS
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    c.customerNumber

ORDENE INVERSAMENTE POR
   customerTotal
```

In [ ]:
query = """
SELECT
    c.customerNumber,
    c.customerName,
    sum(od.priceEach*od.quantityOrdered) AS customerTotal,
    sum(od.quantityOrdered) AS nItems
    
FROM
    orderdetails as od,
    orders as o,
    customers as c
    
WHERE 
    od.orderNumber = o.orderNumber AND
    c.customerNumber = o.customerNumber

GROUP BY
    c.customerNumber

ORDER BY customerTotal DESC

LIMIT 5
"""

queryResult = pd.read_sql_query(query,db)

queryResult

Mostre vendas e valores por mês...

```SQL
JUNTE E COMBINE TODOS AS LINHAS DAS TABELAS
    orderdetails as od,
    orders as o,

PEGUE SÓ LINHAS QUE OBEDEÇAM AO CRITÉRIO
    od.orderNumber = o.orderNumber

MOSTRE AS COLUNAS
    substr(o.orderDate,1,7) as month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal
    sum(od.quantityOrdered) AS nItems,

AGRUPE POR
    month

ORDENE INVERSAMENTE POR
   month
```

In [ ]:
query = """
SELECT
    substr(o.orderDate,1,7) as month,
    sum(od.priceEach*od.quantityOrdered) AS monthTotal,
    sum(od.quantityOrdered) AS nItems

FROM
    orderdetails as od,
    orders as o

WHERE
    od.orderNumber = o.orderNumber

GROUP BY
    month
    
ORDER BY month DESC
"""

queryResult = pd.read_sql_query(query,db)

queryResult

In [ ]:
queryResult.nItems.mean()